# Inkswarm DetectLab — Step Runner Notebook (D-0022)

This notebook lets you execute the pipeline **step-by-step** for **`login_attempt`**, while reusing the existing code structure.

## Steps (run in order, or selectively)
1. **Raw + Dataset** (or reuse)
2. **Features** (or reuse / shared-cache restore)
3. **Baselines** (or reuse artifacts)
4. **Eval** (slice + stability reports)
5. **Export** (summary, UI bundle, handover, evidence)

## Deep dives (optional)
- `02_featurelab_login_attempt.ipynb` — feature exploration
- `03_baselinelab_login_attempt.ipynb` — baseline exploration

---

In [ ]:
from pathlib import Path
import json

from inkswarm_detectlab.config import load_config
from inkswarm_detectlab.ui.steps import StepRecorder
from inkswarm_detectlab.ui.notebook_tools import find_run_dir, print_run_tree, tail_text
from inkswarm_detectlab.ui.step_runner import (
    resolve_run_id,
    wire_check,
    step_dataset,
    step_features,
    step_baselines,
    step_eval,
    step_export,
)

# ==========
# USER INPUT
# ==========

# Pick a config. For a tiny run, use:
#   configs/skynet_smoke.yaml
# For a fuller run:
#   configs/skynet_mvp.yaml
CFG_PATH = Path("configs/skynet_smoke.yaml")

# If None, a run_id is generated from cfg fingerprint.
RUN_ID = None  # e.g. "RUN_SAMPLE_SMOKE_0001"

# Per-step toggles (BQ2=C: explicit manual control)
REUSE_IF_EXISTS = True

DO_STEP_1_DATASET = True
FORCE_STEP_1_DATASET = False

DO_STEP_2_FEATURES = True
FORCE_STEP_2_FEATURES = False
USE_SHARED_FEATURE_CACHE = True
WRITE_SHARED_FEATURE_CACHE = True

DO_STEP_3_BASELINES = True
FORCE_STEP_3_BASELINES = False

DO_STEP_4_EVAL = True
FORCE_STEP_4_EVAL = False

DO_STEP_5_EXPORT = True
FORCE_STEP_5_EXPORT = False

# ==========
# END USER INPUT
# ==========

cfg, run_id = resolve_run_id(CFG_PATH, run_id=RUN_ID)
print("cfg_path:", CFG_PATH)
print("run_id:", run_id)


In [ ]:
# Step 0 — wiring / paths check (Dry-run A)
check = wire_check(CFG_PATH, run_id=run_id)
print(json.dumps(check, indent=2))
rdir = Path(check["paths"]["run_dir"])
print("\nRun tree (quick):")
print_run_tree(rdir)


In [ ]:
# Utility for consistent step prints
def _print_outcome(out):
    print(f"\n=== STEP: {out.name} ===")
    print("status:", out.status)
    print("decision:", f"{out.decision.mode} — {out.decision.reason}")
    if out.notes:
        print("notes:")
        for n in out.notes:
            print(" -", n)
    if out.outputs:
        print("outputs:")
        for k, a in out.outputs.items():
            try:
                p = a.get("path")  # if dict-like
                ex = a.get("exists")
                print(f" - {k}: {p} (exists={ex})")
            except Exception:
                print(f" - {k}: {getattr(a, 'path', '')} (exists={getattr(a, 'exists', None)})")
    if out.summary:
        print("summary:", out.summary)


In [ ]:
# Step 1 — Raw + Dataset (or reuse)
rec = StepRecorder()
out1 = None
if DO_STEP_1_DATASET:
    out1 = step_dataset(
        cfg,
        cfg_path=CFG_PATH,
        run_id=run_id,
        rec=rec,
        reuse_if_exists=REUSE_IF_EXISTS,
        force=FORCE_STEP_1_DATASET,
    )
    _print_outcome(out1)
else:
    print("Skipped (DO_STEP_1_DATASET=False)")


In [ ]:
# Step 1b — Labels + sanity checks (lightweight)
# Surfaces label assignment logic (non-random) and basic distributions.
from inkswarm_detectlab.synthetic.label_defs import as_markdown_table
from inkswarm_detectlab.io.tables import read_auto

rdir = find_run_dir(cfg.paths.runs_dir, run_id)

print("\nLabel definitions (synthetic scenarios):")
print(as_markdown_table())

train_path = (rdir / "dataset" / "login_attempt" / "train.parquet")
if train_path.exists():
    df = read_auto(train_path)
    label_cols = [c for c in df.columns if c.startswith("label_")]
    print("\nTrain split size:", len(df))
    if label_cols:
        print("Label prevalence (train):")
        for c in sorted(label_cols):
            prev = float(df[c].mean())
            print(f" - {c}: {prev:.4f}")
    else:
        print("No label_ columns found in dataset table (unexpected).")
else:
    print("Train split missing; run Step 1 first.")


In [ ]:
# Step 2 — Features (or reuse / shared-cache restore)
out2 = None
if DO_STEP_2_FEATURES:
    out2 = step_features(
        cfg,
        cfg_path=CFG_PATH,
        run_id=run_id,
        rec=rec,
        reuse_if_exists=REUSE_IF_EXISTS,
        force=FORCE_STEP_2_FEATURES,
        use_cache=USE_SHARED_FEATURE_CACHE,
        write_cache=WRITE_SHARED_FEATURE_CACHE,
    )
    _print_outcome(out2)
    print("\nLog tail (featurelab):")
    log_path = Path(cfg.paths.runs_dir) / run_id / "share" / "logs" / "featurelab.log"
    if log_path.exists():
        print(tail_text(log_path, n_lines=120))
    else:
        print("(no featurelab.log yet)")
else:
    print("Skipped (DO_STEP_2_FEATURES=False)")


In [ ]:
# Step 3 — Baselines (or reuse artifacts)
out3 = None
if DO_STEP_3_BASELINES:
    out3 = step_baselines(
        cfg,
        run_id=run_id,
        rec=rec,
        reuse_if_exists=REUSE_IF_EXISTS,
        force=FORCE_STEP_3_BASELINES,
        cfg_path=CFG_PATH,
    )
    _print_outcome(out3)
    # Quick look at metrics.json if present
    metrics_path = Path(cfg.paths.runs_dir) / run_id / "models" / "login_attempt" / "baselines" / "metrics.json"
    if metrics_path.exists():
        metrics = json.loads(metrics_path.read_text(encoding="utf-8"))
        print("\nBaseline summary (meta):")
        print(json.dumps(metrics.get("meta", {}), indent=2))
else:
    print("Skipped (DO_STEP_3_BASELINES=False)")


In [ ]:
# Step 4 — Eval (slice + stability reports)
out4 = None
if DO_STEP_4_EVAL:
    out4 = step_eval(
        cfg,
        cfg_path=CFG_PATH,
        run_id=run_id,
        rec=rec,
        reuse_if_exists=REUSE_IF_EXISTS,
        force=FORCE_STEP_4_EVAL,
    )
    _print_outcome(out4)
    # Point to key report files
    rdir = Path(cfg.paths.runs_dir) / run_id
    for rel in [
        "reports/eval_slices_login_attempt.md",
        "reports/eval_stability_login_attempt.md",
    ]:
        p = rdir / rel
        print(f" - {rel}: {'OK' if p.exists() else 'missing'} ({p})")
else:
    print("Skipped (DO_STEP_4_EVAL=False)")


In [ ]:
# Step 5 — Export share bundle (summary + UI bundle + handover + evidence)
out5 = None
if DO_STEP_5_EXPORT:
    out5 = step_export(
        cfg,
        cfg_path=CFG_PATH,
        run_id=run_id,
        rec=rec,
        reuse_if_exists=REUSE_IF_EXISTS,
        force=FORCE_STEP_5_EXPORT,
    )
    _print_outcome(out5)
    print("\nRun tree (after export):")
    print_run_tree(Path(cfg.paths.runs_dir) / run_id)
else:
    print("Skipped (DO_STEP_5_EXPORT=False)")


In [ ]:
# Step summary table (D-0022 visibility)
print(rec.to_markdown())
